In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
data= fetch_openml('mnist_784', version=1, parser='auto')  # data from https://www.openml.org/d/554
dfData = pd.DataFrame(np.c_[data["data"], data["target"]],
                      columns = data["feature_names"] + ["target"])

In [2]:
stratSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
for train_index, test_index in stratSplit.split(dfData[data["feature_names"]], dfData["target"]):
    X_train = dfData[data["feature_names"]].iloc[train_index]
    X_test = dfData[data["feature_names"]].iloc[test_index]
    
    y_train = dfData["target"].iloc[train_index]
    y_test = dfData["target"].iloc[test_index]

for train_index, val_index in stratSplit.split(X_train, y_train):
    X_train_v = X_train.iloc[train_index]
    X_val = X_train.iloc[val_index]
    
    y_train_v = y_train.iloc[train_index]
    y_val = y_train.iloc[val_index]

In [3]:
log_reg = LogisticRegression()
log_reg.fit(X_train_v, y_train_v)

C:\Users\eequa\Anaconda3\envs\ai\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [4]:
log_reg.score(X_val, y_val)

0.9208928571428572

In [5]:
from sklearn.preprocessing import MinMaxScaler
minMaxScaler = MinMaxScaler()
X_train_v_mm = minMaxScaler.fit_transform(X_train_v.copy())
X_val_mm = minMaxScaler.transform(X_val.copy())
log_reg = LogisticRegression()
log_reg.fit(X_train_v_mm, y_train_v)
log_reg.score(X_val_mm, y_val)

C:\Users\eequa\Anaconda3\envs\ai\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9224107142857143

In [6]:
from sklearn.preprocessing import StandardScaler
stdScaler = StandardScaler()
X_train_v_sc = stdScaler.fit_transform(X_train_v.copy())
X_val_sc = stdScaler.transform(X_val.copy())
log_reg = LogisticRegression()
log_reg.fit(X_train_v_sc, y_train_v)
log_reg.score(X_val_sc, y_val)

C:\Users\eequa\Anaconda3\envs\ai\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9132142857142858

In [7]:
def logValues(val):
    if val == 0:
        return np.log(1)
    return np.log(val)

X_train_log = X_train_v.copy()
X_val_log = X_val.copy()
for col in X_train_log.columns:
    X_train_log[col] = X_train_log[col].apply(logValues)
    X_val_log[col] = X_val_log[col].apply(logValues)
log_reg = LogisticRegression()
log_reg.fit(X_train_log, y_train_v)
log_reg.score(X_val_log, y_val)

C:\Users\eequa\Anaconda3\envs\ai\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9198214285714286

In [8]:
X_train_perc = X_train_v.copy()
X_val_perc = X_val.copy()

for col in X_train_perc.columns:
    X_train_perc[col] = X_train_perc[col].rank(pct=True)
    X_val_perc[col] = X_val_perc[col].rank(pct=True)
    
log_reg = LogisticRegression()
log_reg.fit(X_train_perc, y_train_v)
log_reg.score(X_val_perc, y_val)

C:\Users\eequa\Anaconda3\envs\ai\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9182142857142858

In [9]:
def applyThresh(val,threshold=0.5):
    if val < threshold:
        return 0
    return 1

X_train_thresh = X_train_v.copy()
# only using thresholds calculated from training datasets
meanThresh = X_train_v.mean()
X_val_thresh = X_val.copy()
for col in X_train_thresh.columns:
    X_train_thresh[col] = X_train_thresh[col].apply(applyThresh, threshold=int(meanThresh[col]))
    X_val_thresh[col] = X_val_thresh[col].apply(applyThresh, threshold=int(meanThresh[col]))
log_reg = LogisticRegression()
log_reg.fit(X_train_thresh, y_train_v)
log_reg.score(X_val_thresh, y_val)

C:\Users\eequa\Anaconda3\envs\ai\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9178571428571428